Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All) to avoid typical problems with Jupyter notebooks. **Unfortunately, this does not work with Chrome right now, you will also need to reload the tab in Chrome afterwards**.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Please put your name here:

In [473]:
NAME = "AVISHA BHIRYANI"

---

# Evaluation

In this (shorter) assignment, we want to compare the quality of different clustering approaches.

In [474]:
import numpy as np, pandas as pd, scipy

In [475]:
# Load the input data
import json, gzip
raw = json.load(gzip.open("/data/simpsonswiki.json.gz", "rt", encoding="utf-8"))
titles, texts, classes = [x["title"] for x in raw], [x["text"] for x in raw], [x["c"] for x in raw]

This is a minimal example implementation of spherical k-means, which we will use in the following.

In [476]:
# Vectorize the text for k-means (minimalistic)
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(stop_words="english", sublinear_tf=True, smooth_idf=False, min_df=5)
vect.fit(texts)
vect.idf_ -= 1
tfidf, idf = vect.transform(texts), vect.idf_
vocabulary = vect.get_feature_names_out()

In [477]:
# Minimalistic implementation for spherical k-means, so we use the same version in this assignment
# This is NOT meant as an example of good code, but to be snort.
def initial_centers(tfidf, k, seed):
    return tfidf[np.random.default_rng(seed=seed).choice(tfidf.shape[0], k, replace=False)]

def sphericalkmeans(tfidf, centers, max_iter=100):
    """REQUIRES the input to be L2 normalized, and does not handle corner cases such as empty clusters!"""
    last_assignment = None
    for iter in range(max_iter):
        assignment = np.asarray((tfidf @ centers.T).argmax(axis=1)).squeeze()
        if last_assignment is not None and all(assignment == last_assignment): break
        last_assignment, centers = assignment, np.zeros(centers.shape)
        for i in range(centers.shape[0]):
            c = tfidf[assignment == i,:].sum(axis=0).A1
            centers[i] = c / np.sqrt((c**2).sum())
    return centers, assignment, iter

## Implement a function to compute a cross-tabulation matrix

Compute the cross-tabulation matrix compares every class to every cluster. Append an additional row and column for the cluster sizes / class totals and the dataset size. Make sure to accept clusters that are, e.g., labeled using text labels and *not* just as integers 0..k.

Write your own code, do not use `pandas.crosstab`.

You do not need to vectorize this, but try to use numpy operations where easily possible - in particular if you end up waiting a lot for results below!

In [478]:
from collections import Counter
def cross_tabulation(clu, cla):
    """Compute the cross-tabulation matrix to compare assignments `clu` and `cla`."""
    # YOUR CODE HERE
    clusters = list(set(clu))
    classes = list(set(cla))
    
    classes_dict = {}
    i = 0
    for class_val in classes:
        classes_dict[class_val] = i
        i = i +1
        
    classes_transformed = [classes_dict.get(x) for x in cla]

    cross_tab = np.full((len(clusters)+1,len(classes)+1), int(0))
    dict_cluster_points = {}
    
    for i in clusters:
        dict_cluster_points[i] = np.flatnonzero((clu == i))
        class_mapping = np.array(classes_transformed)[dict_cluster_points[i].astype(int)]
        cluster_class_count = dict(Counter(class_mapping))

        for key, value in cluster_class_count.items():
            cross_tab[i,int(key)] = int(value)
    
    cross_tab[-1] = np.sum(cross_tab, axis=0)
    cross_tab[:, -1] = np.sum(cross_tab, axis=1)
    
    return cross_tab 

In [479]:
_, _tmp, _ = sphericalkmeans(tfidf, tfidf[:5], 1)


In [480]:
cross_tabulation(_tmp, classes)

array([[  435,   277,    74,   671,  2657,   220,   870,    33,   516,
         5753],
       [  141,    88,     9,   132,   726,    24,   293,     8,   212,
         1633],
       [   64,    35,     3,   176,   474,     4,   252,     4,   168,
         1180],
       [   44,    18,     1,    44,   346,     5,   125,     3,    59,
          645],
       [   45,    47,     7,    70,   367,    19,   230,     5,   125,
          915],
       [  729,   465,    94,  1093,  4570,   272,  1770,    53,  1080,
        10126]])

In [481]:
### Automatic tests
_, _tmp, _ = sphericalkmeans(tfidf, tfidf[:5], 1)
_tmp2 = cross_tabulation(_tmp, np.ones(tfidf.shape[0]))
assert isinstance(_tmp2, np.ndarray), "Must be an array"
assert np.issubdtype(_tmp2.dtype, np.integer), "Must be an integer array"
if _tmp2.shape == (2,6): print("Use first parameter as first index.")
assert _tmp2.shape == (6,2), "Wrong shape"
assert _tmp2[:-1,:-1].sum() == tfidf.shape[0], "Not all elements"
assert (_tmp2[:,:-1].sum(axis=1) == _tmp2[:,-1]).all(), "Sizes are bad"
assert (_tmp2[:-1].sum(axis=0) == _tmp2[-1]).all(), "Sizes are bad"
assert _tmp2.sum() == 4 * tfidf.shape[0], "Not all elements"
from unittest.mock import patch
with patch('pandas.crosstab', side_effect=Exception("Do not use pandas.crosstab")) as mock_p:
    _tmp2 = cross_tabulation(_tmp, _tmp)
assert _tmp2.shape == (6,6), "Wrong shape"
assert _tmp2.sum() == 4 * tfidf.shape[0], "Not all elements"
assert (_tmp2[:,:-1].sum(axis=1) == _tmp2[:,-1]).all(), "Sizes are bad"
assert (_tmp2[:-1].sum(axis=0) == _tmp2[-1]).all(), "Sizes are bad"
del _tmp, _tmp2

In [482]:
### Automatic tests
_, _tmp, _ = sphericalkmeans(tfidf, tfidf[:5], 1)
try:
    _tmp2 = cross_tabulation(_tmp, classes)
except Exception as e:
    raise Exception("Your code probably does not accept textual class labels.")
assert _tmp2.shape == (6,len(np.unique(classes))+1), "Wrong shape"
assert _tmp2.sum() == 4 * tfidf.shape[0], "Not all elements"
assert (_tmp2[:,:-1].sum(axis=1) == _tmp2[:,-1]).all(), "Sizes are bad"
assert (_tmp2[:-1].sum(axis=0) == _tmp2[-1]).all(), "Sizes are bad"
del _tmp, _tmp2

## Implement a function to compute the pair counts from the cross-tabulation matrix

In [483]:
from math import comb
def pair_count(crosstab):
    """Compute the pair count matrix from the cross-tabulation matrix."""
    # YOUR CODE HERE
    data = crosstab[:-1, :-1]
    pairs_data = np.full((data.shape[0]+1,data.shape[1]+1), int(0))
    
    for i in range(len(data)):
        for j in range(len(data[i])):
            pairs_data[i][j] = comb(data[i][j],2)
  
    pairs_clusters = 0
    for i in range(data.shape[0]):
        pairs_data[i,-1] = comb(np.sum(data[i]),2)
    
    for i in range(len(crosstab[-1])):
        pairs_data[-1,i] = comb(crosstab[-1,i],2) 
    return pairs_data   

In [484]:
def pair_count_mat(crosstab):
    """Compute the pair count matrix from the cross-tabulation matrix."""
    # YOUR CODE HERE
    data = crosstab[:-1, :-1]
    pairs_data = np.full((data.shape[0],data.shape[1]), int(0))
    
    for i in range(len(data)):
        for j in range(len(data[i])):
            pairs_data[i][j] = comb(data[i][j],2)
    true_positive = pairs_data.sum()
    
    pairs_clusters = 0
    for i in range(data.shape[0]):
        pairs_clusters = pairs_clusters + comb(np.sum(pairs_data[i]),2)
    false_positive = pairs_clusters - true_positive
    
    last_row = crosstab[-1,:]
    pairs_class = 0
    for i in range(len(last_row)-1):
        pairs_class = pairs_class + comb(last_row[i],2)
    false_negative = pairs_class - true_positive
    
    true_negative = comb(crosstab[-1,-1],2) - true_positive - false_positive - false_negative
    
    pair_count_mat = np.full((2,2),int(0))
    pair_count_mat[0,0], pair_count_mat[0,1] = true_positive, false_positive
    pair_count_mat[1,0], pair_count_mat[1,1] = false_negative, true_negative
    
    return pair_count_mat

In [485]:
### Automatic tests
_, _tmp, _ = sphericalkmeans(tfidf, tfidf[:5], 1)
_tmp2 = pair_count(cross_tabulation(_tmp, np.ones(tfidf.shape[0])))
assert isinstance(_tmp2, np.ndarray), "Must be an array"
assert np.issubdtype(_tmp2.dtype, np.integer), "Must be an integer array"
if _tmp2.shape == (2,6): print("Use first parameter as first index.")
assert _tmp2.shape == (6,2), "Wrong shape"
assert (_tmp2[:,:-1].sum(axis=1) == _tmp2[:,-1]).all(), "Wrong result"
assert not (_tmp2[:-1].sum(axis=0) == _tmp2[-1]).all(), "Wrong result"
_tmp2 = cross_tabulation(_tmp, _tmp)
assert _tmp2.shape == (6,6), "Wrong shape"
assert (_tmp2[:,:-1].sum(axis=1) == _tmp2[:,-1]).all(), "Wrong result"
assert (_tmp2[:-1].sum(axis=0) == _tmp2[-1]).all(), "Wrong result"
del _tmp, _tmp2

## Compute the Rand Index

First compute the Rand Index of two assignments. You must use above functions.

In [486]:
def rand_index(clu, cla):
    # YOUR CODE HERE
    crosstab = cross_tabulation(clu, cla)
    pairs_data = pair_count(crosstab)
    print(pairs_data)
    data = pairs_data[:-1, :-1]
    true_positive = data.sum()
    false_positive = np.sum(pairs_data[:,-1]) - pairs_data[-1,-1] - true_positive
    false_negative = np.sum(pairs_data[-1]) - pairs_data[-1,-1] - true_positive
    true_negative = pairs_data[-1,-1] - true_positive - false_positive - false_negative
    index = (true_positive + true_negative) / pairs_data[-1,-1]
    return index
    

In [487]:
### Automatic tests
_, _tmp, _ = sphericalkmeans(tfidf, tfidf[:5], 1)
from unittest.mock import patch
with patch('__main__.cross_tabulation') as mock_u1, patch('__main__.pair_count') as mock_u2, patch('sklearn.metrics.rand_score') as mock_sk:
    rand_index(_tmp, classes)
assert mock_u1.called, "Use the cross_tabulation function!"
assert mock_u2.called, "Use the pair_count function!"
assert not mock_sk.called, "Use your own code, not sklearn."
ri = rand_index(_tmp, classes)
assert ri <= 1, "RI must be at most 1."
assert ri > 0, "RI must be always larger than 0."

<MagicMock name='pair_count()' id='139704747577456'>
[[   94395    38226     2701   224785  3528496    24090   378015      528
    132870 16545628]
 [    9870     3828       36     8646   263175      276    42778       28
     22366  1332528]
 [    2016      595        3    15400   112101        6    31626        6
     14028   695610]
 [     946      153        0      946    59685       10     7750        3
      1711   207690]
 [     990     1081       21     2415    67161      171    26335       10
      7750   418155]
 [  265356   107880     4371   596778 10440165    36856  1565565     1378
    582660 51262875]]


In [488]:
### Automatic tests
import sklearn.metrics
assert abs(ri - sklearn.metrics.rand_score(_tmp, classes)) < 1e-7, "Result should agree with sklearn"
del _tmp

## Compute the Adjusted Rand Index

Write a function to compute the adjusted Rand index of two assignments. You must use above `pair_count` and `cross_tabulation` functions.

Beware of integer overflows when using the equation from the slides. To resolve the integer overflow, transform the equation such that it has the standard form $ARI = \frac{RI-E[RI]}{M-E[RI]}$ where RI is the rand index, $E[RI]$ is the expected value of the rand index (you need to derive this from the ARI equation given on the slides, do *not* attempt to figure out this equation directly; this assignment only needs standad high school math), and \(M\) is the maximum possible value of the Rand index (a constant).

In [489]:
def adjusted_rand_index(clu, cla):
    # YOUR CODE HERE
    crosstab = cross_tabulation(clu, cla)
    pairs_data = pair_count(crosstab)
    data = pairs_data[:-1, :-1]
    
    true_positive = data.sum()
    false_positive = np.sum(pairs_data[:,-1]) - pairs_data[-1,-1] - true_positive
    false_negative = np.sum(pairs_data[-1]) - pairs_data[-1,-1] - true_positive
    true_negative = pairs_data[-1,-1] - true_positive - false_positive - false_negative
    
    # array to resolve overflow error
    v1_1 = int(true_positive+false_positive)
    v1_2 = int(true_positive+false_negative)
    v1 = int(v1_1*v1_2)
    v2_1= int(false_negative+true_negative)
    v2_2 = int(false_positive+true_negative)
    v2 = int(v2_1 * v2_2)
    expected_index = int(v1+v2)
    index = int((true_positive + true_negative) * pairs_data[-1,-1])
    optimal = np.power(pairs_data[-1,-1],2).astype(int)
    
    adj_index = (index - expected_index)/(optimal-expected_index)
    return adj_index

In [490]:
### Automatic tests
_, _tmp, _ = sphericalkmeans(tfidf, tfidf[:5], 1)
from unittest.mock import patch
with patch('__main__.cross_tabulation') as mock_u1, patch('__main__.pair_count') as mock_u2, patch('sklearn.metrics.adjusted_rand_score') as mock_sk, patch('sklearn.metrics.rand_score') as mock_sk2:
    adjusted_rand_index(_tmp, classes)
assert mock_u1.called, "Use the cross_tabulation function!"
assert mock_u2.called, "Use the pair_count function!"
assert not mock_sk.called, "Use your own code, not sklearn."
assert not mock_sk2.called, "Use your own code, not sklearn."
ari = adjusted_rand_index(_tmp, classes)
assert ari <= 1, "ARI must be at most 1"
assert ari > 0, "This clustering must score better than random."
del _tmp

In [491]:
### Automatic tests
_, _tmp, _ = sphericalkmeans(tfidf, tfidf[:5], 1)
ari = adjusted_rand_index(_tmp, classes)
import sklearn.metrics
assert abs(ari - sklearn.metrics.adjusted_rand_score(_tmp, classes)) < 1e-7, "Result should agree with sklearn"
del _tmp

## Compute the Normalized Mutual Information

Write a function to compute the Normalized Mutual Information (with arithmetic averaging) of two assignments.
You must use above `pair_count` and `cross_tabulation` functions.

In [492]:
from math import log2
def normalized_mutual_information(clu, cla):
    # YOUR CODE HERE
    crosstab = cross_tabulation(clu, cla)
    pairs_data = pair_count(crosstab)
    N = crosstab[-1,-1]
    information = 0
    entropy1 = 0
    entropy2 = 0
    
    
    
    for i in range(len(crosstab)-1):
        for j in range(len(crosstab[i])-1):
            val = (N*crosstab[i][j])/(crosstab[i,-1]*crosstab[-1,j])
            if val>0:
                information = information + ((crosstab[i][j]/N)*(np.log(val)))
    arr = crosstab[:,-1]
    for i in range(len(arr)-1):
        entropy1 = entropy1 + -(arr[i]/N)*(np.log(arr[i]/N)) 
    
    arr = crosstab[-1]
    for i in range(len(arr)-1):
        entropy2 = entropy2 + -(arr[i]/N)*(np.log(arr[i]/N)) 
    
    nmi = (2*information)/(entropy1+entropy2)
    return nmi

In [493]:
### Automatic tests
_, _tmp, _ = sphericalkmeans(tfidf, tfidf[:5], 1)
from unittest.mock import patch
with patch('sklearn.metrics.normalized_mutual_info_score', side_effect=Exception("Use your own code, not skelarn")) as mock_sk1,  patch('sklearn.metrics.mutual_info_score', side_effect=Exception("Use your own code, not skelarn")) as mock_sk2:
    normalized_mutual_information(_tmp, classes)
nmi = normalized_mutual_information(_tmp, classes)
assert nmi <= 1, "NMI must be at most 1"
assert nmi > 0, "This clustering must score better than random."
del _tmp

In [494]:
### Automatic tests
_, _tmp, _ = sphericalkmeans(tfidf, tfidf[:5], 1)
nmi = normalized_mutual_information(_tmp, classes)
import sklearn.metrics
assert abs(nmi - sklearn.metrics.normalized_mutual_info_score(_tmp, classes)) < 1e-10, "Result should agree with sklearn"
del _tmp

## Finding the best clustering

for $k=1..15$, and a fixed random seed of 0, find the best spherical k-means clustering by NMI compared to the classes stored in `classes` above (note that this will not generally be possible, as our data usually will not be labeled).

In [495]:
bestk = None # Store best k here
bestnmi = None # Store the best NMI here
bestassignment = None # Store the best assignment here
# YOUR CODE HERE
raise NotImplementedError()
print("The best k is", bestk, "scoring", bestnmi)
# Hint: it will *not* score very good. The classes are not clusters.

NotImplementedError: 

In [ ]:
### Automatic tests
assert bestk > 0 and bestk <= 15
assert bestk == len(np.unique(bestassignment)), "Inconsistent result"
assert abs(bestnmi-sklearn.metrics.normalized_mutual_info_score(bestassignment, classes)) < 1e-7, "Inconsistent result"
assert all(bestassignment == sphericalkmeans(tfidf, initial_centers(tfidf, bestk, 0))[1]), "Inconsistent result"
assert bestk > 2, "There should be something better"
assert bestk < 15, "There should be something better"

In [ ]:
### Additional Automatic tests

## Explore the result

Explore the clustering result, by comparing it to the original classes.

For each cluster, return the cluster label, the three top classes, and the percentages of the clusters.

In [ ]:
def top_classes(clu, cla):
    """For each cluster, give the top three classes and their share of the data each."""
    # For each cluster, call yield label, *top3, *shares to return a 7-tuple.
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
### Automatic tests
_tmp = top_classes(classes, classes)
import types
assert isinstance(_tmp, types.GeneratorType), "You did not use `yield`."
_tmp = list(_tmp)
assert len(_tmp) == len(np.unique(classes)), "Wrong number of results"
for row in _tmp:
    assert len(row) == 7, "Not a 7-tuple"
    assert row[0] in classes, "Not a label"
    assert row[0] == row[1], "Incorrect result"
    assert row[4:] == (1.,0.,0.), "Incorrect result"
del _tmp

In [ ]:
### Automatic tests
_tmp = top_classes(bestassignment, classes)
import types
assert isinstance(_tmp, types.GeneratorType), "You did not use `yield`."
_tmp = list(_tmp)
assert len(_tmp) == bestk, "Wrong number of results"
for row in _tmp:
    assert len(row) == 7, "Not a 7-tuple"
    assert row[1] in classes, "Not a label"
# Additional hidden tests
del _tmp

In [ ]:
# Explore your best clustering!
# YOUR CODE HERE
raise NotImplementedError()